# Alignment Error Visualization

This notebook collects COM data from the database and tries to quantify some alignment errors. The main results are shown in the plots at the end of the notebook.

In [1]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from collections import OrderedDict
from IPython.display import HTML
from itertools import combinations
from scipy.ndimage import affine_transform


PIPELINE_ROOT = Path('./').absolute().parents[1]
PIPELINE_ROOT = PIPELINE_ROOT.as_posix()
sys.path.append(PIPELINE_ROOT)
print(PIPELINE_ROOT)


/home/eddyod/programming/pipeline/src


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from library.controller.sql_controller import SqlController
from library.image_manipulation.filelocation_manager import FileLocationManager
from library.atlas.atlas_utilities import apply_affine_transform, get_affine_transformation, \
fetch_coms, list_coms, compute_affine_transformation
from library.atlas.brain_structure_manager import BrainStructureManager
from library.utilities.utilities_process import M_UM_SCALE, SCALING_FACTOR, random_string, \
read_image, write_image


XGBoost Version: 2.1.4


In [108]:
def sum_square_com(com):
    ss = np.sqrt(sum([s**2 for s in com]))
    return ss

def apply_affine_transformation(volume, matrix):
    """Apply an affine transformation to a 3D volume."""
    transformed_volume = affine_transform(volume, matrix, offset=0, order=1)
    return transformed_volume

import numpy as np
from scipy.spatial.transform import Rotation as R

def euler_to_rigid_transform(euler_transform, rotation_order='xyz', degrees=False):
    """
    Converts a 6-variable Euler transform to a 4x4 rigid transformation matrix.
    
    Parameters:
        euler_transform (list or np.ndarray): A list or array of 6 values.
                                              The first 3 are rotation (rx, ry, rz),
                                              the last 3 are translation (tx, ty, tz).
        rotation_order (str): Order of Euler rotations (default 'xyz').
        degrees (bool): Whether the input rotation angles are in degrees. Default is radians.

    Returns:
        np.ndarray: A 4x4 rigid transformation matrix.
    """
    assert len(euler_transform) == 6, "Euler transform must have 6 elements"

    rot_angles = euler_transform[:3]
    translation = euler_transform[3:]

    # Create rotation matrix
    rotation = R.from_euler(rotation_order, rot_angles, degrees=degrees)
    rot_matrix = rotation.as_matrix()

    # Construct 4x4 transformation matrix
    transform = np.eye(4)
    transform[:3, :3] = rot_matrix
    transform[:3, 3] = translation

    return transform
# Example usage:


In [41]:
moving_name = 'DK55'
fixed_name = 'Allen'

moving_all = list_coms(moving_name, scaling_factor=10)
fixed_all = list_coms(fixed_name, scaling_factor=1)

common_keys = list(moving_all.keys() & fixed_all.keys())
print(f'{moving_name} len={len(moving_all.keys())}')
print(f'{fixed_name} len={len(fixed_all.keys())}')

print(len(common_keys))

DK55 len=28
Allen len=37
22


In [42]:
# these keys have very high errors: 'RtTg', 'AP'
midbrain_keys = {
            "3N_L",
            "3N_R",
            "4N_L",
            "4N_R",
            "IC",
            "PBG_L",
            "PBG_R",
            "SC",
            "SNC_L",
            "SNC_R",
            "SNR_L",
            "SNR_R",
        }
bad_keys = ('RtTg', 'AP', '3N_L', '3N_R')
bad_keys = ('RtTg', 'AP')
#bad_keys = ('RtTg',)
bad_keys = ()
good_keys = set(common_keys) - set(bad_keys)
print(f'#good_keys={len(good_keys)}')
print(good_keys)

#good_keys=22
{'DC_L', 'LRt_L', 'LRt_R', '3N_R', '6N_R', 'Amb_R', 'PBG_L', '4N_R', 'DC_R', 'LC_R', '3N_L', '7N_R', '5N_L', 'LC_L', '6N_L', '7n_L', 'PBG_R', '4N_L', 'Amb_L', '7N_L', '5N_R', 'AP'}


In [114]:
#moving_src = np.array([moving_all[s] for s in good_keys])
#fixed_src = np.array([fixed_all[s] for s in good_keys])
#transformation_matrix = compute_affine_transformation(moving_src, fixed_src)

transform_parameters = [0.955146, 0.333175, 0.110121, -0.462277, 0.956330, 
                                    0.056532, 0.003245, -0.017109, 0.969857, 307.262939, 123.364020, -40.399677]
transform_parameters = [0.955146, 0.333175, 0.110121, -0.462277, 0.956330, 
                                    0.056532, 0.003245, -0.017109, 0.969857, 0, 0, 0]
rotation = transform_parameters[:9]
rotation = np.array(rotation).reshape(3,3)
transformation_matrix = rotation.copy()
translation = np.array(transform_parameters[-3:])
t = translation.reshape(3,1)
transformation_matrix = np.hstack( [rotation, t ])
transformation_matrix = np.vstack([transformation_matrix, np.array([0, 0, 0, 1])])
#print(repr(transformation_matrix))

euler = [-0.070265, -0.021537, -0.490839, 126.889695, 44.602420, -18.971854]  # [rx, ry, rz, tx, ty, tz]
T = euler_to_rigid_transform(euler, degrees=False)
#print("Rigid Transformation Matrix:\n", T)
print(repr(T))
A = T[:3, :3]
print(A.shape)

array([[ 8.81733161e-01,  4.71536304e-01,  1.41473275e-02,
         1.26889695e+02],
       [-4.71256684e-01,  8.79048775e-01,  7.20443488e-02,
         4.46024200e+01],
       [ 2.15353351e-02, -7.01909140e-02,  9.97301090e-01,
        -1.89718540e+01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])
(3, 3)


In [117]:
# Apply affine transformation
# Apply affine transformation
origin = np.array([535, 666, 515])
print(origin)
#print(origin + translation/10)
#transformation_matrix = np.linalg.inv(transformation_matrix)
trans_origin = apply_affine_transform(origin, T)
print(np.round(trans_origin))
# 100,10
#[884 697 296]
#[718. 778. 224.]

[535 666 515]
[920. 415. 459.]


In [39]:
df_list = []
error = []
transformed_dict = {}
for structure in common_keys:
    moving0 = np.array(moving_all[structure])
    fixed0 = np.array(fixed_all[structure]) 
    transformed = apply_affine_transform(moving0, transformation_matrix)
    transformed = [x for x in transformed]
    difference = [a - b for a, b in zip(transformed, fixed0)]
    ss = sum_square_com(difference)
    row = [structure, np.round(moving0), np.round(fixed0), 
           np.round(transformed), np.round(difference), ss]
    df_list.append(row)
    error.append(ss)
    transformed_dict[structure] = transformed
print('RMS', sum(error)/len(df_list))
# MD589 to Allen RMS 260.0211852431133
# MD585 to Allen RMS 263.314352291951
# MD594 to Allen RMS 250.79820210419254
# AtlasV8 disk to Allen RMS 237.0680595008574
# MD585 to MD589 RMS 18.2658167690059

RMS 244.05006368736576


In [40]:
structure = 'SC'
try:
    com = moving_all[structure]
except KeyError:
    structure = common_keys[0]
    com = moving_all[structure]
transformed_structure = apply_affine_transform(com, transformation_matrix)
print(f'{moving_name} {structure} {np.round(np.array(com))}')
print(f'{fixed_name} {structure} {np.round(np.array(fixed_all[structure]))}')
print(f'{moving_name} transformed {structure} {np.round(np.array(transformed_dict[structure]))}')
#print(f'neuro res {moving_name} {structure} {np.round(com)}')


AtlasV8 SC [8345. 3155. 6919.]
Allen SC [9140. 2388. 5692.]
AtlasV8 transformed SC [9144. 2216. 5742.]


In [10]:
columns = ['structure', moving_name, fixed_name, 'transformed', 'difference', 'sumsquares']
df = pd.DataFrame(df_list, columns=columns)
df.index.name = 'Index'
df = df.round(4)
df.sort_values(by=['sumsquares'], inplace=True)
HTML(df.to_html(index=False))

structure,MD589,Allen,transformed,difference,sumsquares
IC,"[12363.0, 4873.0, 4320.0]","[10400.0, 2325.0, 5675.0]","[10379.0, 2299.0, 5640.0]","[-21.0, -26.0, -35.0]",48.3085
PBG_L,"[11184.0, 6169.0, 2560.0]","[9403.0, 3844.0, 3551.0]","[9334.0, 3787.0, 3615.0]","[-69.0, -57.0, 64.0]",110.0859
VLL_L,"[11083.0, 7204.0, 2960.0]","[9464.0, 5176.0, 3890.0]","[9419.0, 5086.0, 3980.0]","[-46.0, -90.0, 90.0]",134.9123
PBG_R,"[11258.0, 5836.0, 6320.0]","[9401.0, 3846.0, 7833.0]","[9527.0, 3833.0, 7770.0]","[126.0, -14.0, -63.0]",142.0177
7N_R,"[12295.0, 8389.0, 5900.0]","[10853.0, 6775.0, 7045.0]","[10949.0, 6700.0, 7119.0]","[96.0, -75.0, 74.0]",142.8741
SNC_R,"[10009.0, 6740.0, 5840.0]","[8356.0, 5123.0, 7092.0]","[8408.0, 5015.0, 7180.0]","[52.0, -108.0, 88.0]",148.4992
4N_L,"[11404.0, 6145.0, 4180.0]","[9589.0, 3778.0, 5432.0]","[9623.0, 3925.0, 5396.0]","[34.0, 147.0, -36.0]",155.6349
5N_R,"[11625.0, 7238.0, 5980.0]","[10193.0, 5286.0, 7293.0]","[10101.0, 5420.0, 7293.0]","[-92.0, 134.0, 0.0]",163.0793
7N_L,"[12338.0, 8595.0, 3460.0]","[10853.0, 6775.0, 4340.0]","[10914.0, 6647.0, 4424.0]","[61.0, -128.0, 84.0]",165.0618
Sp5I_L,"[13699.0, 8058.0, 2700.0]","[12107.0, 5872.0, 3748.0]","[12145.0, 5740.0, 3625.0]","[39.0, -132.0, -123.0]",184.4487
